In [5]:
!pip install pandas

In [7]:
import pandas as pd

foxconndf= pd.read_csv('./0050.csv', index_col=0 )
foxconndf.dropna(how='any',inplace=True)#刪除空值
#print(foxconndf)
from sklearn import preprocessing
def normalize(df):#標準化->把data 都轉成0~1
    newdf= df.copy()
    
    min_max_scaler = preprocessing.MinMaxScaler()
    for feature in df.columns:
        if not feature == 'Date':
            newdf[feature] = min_max_scaler.fit_transform(df[feature].values.reshape(-1,1))

    
    return newdf
foxconndf_norm= normalize(foxconndf)
#print(foxconndf_norm)
import numpy as np
def data_helper(df, time_frame): #把data 分成train(x,y) & text(x,y)
    
    number_features = len(df.columns) # 71個x
    
    datavalue = df.as_matrix()#轉成矩陣
    result = []
    
    for index in range( len(datavalue) - (time_frame+1) ): #前五天預測第六天
        result.append(datavalue[index: index + (time_frame+1) ]) 
    
    result = np.array(result) #變成矩陣
    number_train = round(0.9 * result.shape[0]) 
    
    x_train = result[:int(number_train), :-1] 
    y_train = result[:int(number_train), -1][:,-1] 
    
    
    x_test = result[int(number_train):, :-1]
    y_test = result[int(number_train):, -1][:,-1]
    
    
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], number_features))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], number_features))  
    return [x_train, y_train, x_test, y_test]

X_train, y_train, X_test, y_test = data_helper(foxconndf_norm, 5)

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
import keras
def build_model(input_length, input_dim):
    d = 0.3
    model = Sequential()
    model.add(LSTM(256, input_shape=(input_length, input_dim), return_sequences=True))
    model.add(Dropout(d))
    model.add(LSTM(256, input_shape=(input_length, input_dim), return_sequences=False))
    model.add(Dropout(d))
    model.add(Dense(16,kernel_initializer="uniform",activation='relu'))
    model.add(Dense(1,kernel_initializer="uniform",activation='linear'))
    model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
    return model

model = build_model( 5, 71 )
print(model)
model.fit( X_train, y_train, batch_size=128, epochs=300, validation_split=0.1, verbose=1)
a = model.get_config()
print(a)

def denormalize(df, norm_value):
    original_value = df['close'].values.reshape(-1,1)
    norm_value = norm_value.reshape(-1,1)
    
    min_max_scaler = preprocessing.MinMaxScaler()
    min_max_scaler.fit_transform(original_value)
    denorm_value = min_max_scaler.inverse_transform(norm_value)
    
    return denorm_value

pred = model.predict(X_test)

denorm_pred = denormalize(foxconndf, pred)
denorm_ytest = denormalize(foxconndf, y_test)

import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(denorm_pred,color='red', label='Prediction')
plt.plot(denorm_ytest,color='blue', label='Answer')
plt.legend(loc='best')
plt.show()

C:\Users\Big data\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Big data\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 734 samples, validate on 82 samples
Epoch 1/300
734/734 [==============================] - 1s 2ms/step - loss: 0.3903 - acc: 0.0014 - val_loss: 0.1696 - val_acc: 0.0000e+00
Epoch 2/300
734/734 [==============================] - 0s 428us/step - loss: 0.1132 - acc: 0.0014 - val_loss: 0.0826 - val_acc: 0.0000e+00
Epoch 3/300
734/734 [==============================] - 0s 432us/step - loss: 0.0751 - acc: 0.0014 - val_loss: 0.0683 - val_acc: 0.0000e+00
Epoch 4/300
734/734 [==============================] - 0s 444us/step - loss: 0.0818 - acc: 0.0027 - val_loss: 0.0531 - val_acc: 0.0000e+00
Epoch 5/300
734/734 [==============================] - 0s 466us/step - loss: 0.0644 - acc: 0.0027 - val_loss: 0.0417 - val_acc: 0.0000e+00
Epoch 6/300
734/734 [==============================] - 0s 435us/step - loss: 0.0570 - acc: 0.0027 - val_loss: 0.0341 - val_acc: 0.0000e+00
Epoch 7/300
734/734 [==============================] - 0s 421us/step - loss: 0.0437 - acc: 0.0027 - val_loss: 0.0322 - val_

734/734 [==============================] - 0s 460us/step - loss: 0.0014 - acc: 0.0027 - val_loss: 0.0016 - val_acc: 0.0000e+00
Epoch 59/300
734/734 [==============================] - 0s 496us/step - loss: 0.0015 - acc: 0.0027 - val_loss: 7.1177e-04 - val_acc: 0.0000e+00
Epoch 60/300
734/734 [==============================] - 0s 444us/step - loss: 0.0014 - acc: 0.0027 - val_loss: 0.0025 - val_acc: 0.0000e+00
Epoch 61/300
734/734 [==============================] - 0s 434us/step - loss: 0.0017 - acc: 0.0027 - val_loss: 7.4028e-04 - val_acc: 0.0000e+00
Epoch 62/300
734/734 [==============================] - 0s 432us/step - loss: 0.0016 - acc: 0.0027 - val_loss: 0.0013 - val_acc: 0.0000e+00
Epoch 63/300
734/734 [==============================] - 0s 442us/step - loss: 0.0018 - acc: 0.0027 - val_loss: 0.0016 - val_acc: 0.0000e+00
Epoch 64/300
734/734 [==============================] - 0s 406us/step - loss: 0.0015 - acc: 0.0027 - val_loss: 0.0014 - val_acc: 0.0000e+00
Epoch 65/300
734/734 [===

734/734 [==============================] - 0s 467us/step - loss: 0.0012 - acc: 0.0027 - val_loss: 6.5353e-04 - val_acc: 0.0000e+00
Epoch 116/300
734/734 [==============================] - 0s 437us/step - loss: 0.0013 - acc: 0.0027 - val_loss: 0.0021 - val_acc: 0.0000e+00
Epoch 117/300
734/734 [==============================] - 0s 426us/step - loss: 0.0013 - acc: 0.0027 - val_loss: 7.6944e-04 - val_acc: 0.0000e+00
Epoch 118/300
734/734 [==============================] - 0s 412us/step - loss: 0.0012 - acc: 0.0027 - val_loss: 4.8574e-04 - val_acc: 0.0000e+00
Epoch 119/300
734/734 [==============================] - 0s 392us/step - loss: 0.0011 - acc: 0.0027 - val_loss: 0.0013 - val_acc: 0.0000e+00
Epoch 120/300
734/734 [==============================] - 0s 402us/step - loss: 0.0012 - acc: 0.0027 - val_loss: 6.8844e-04 - val_acc: 0.0000e+00
Epoch 121/300
734/734 [==============================] - 0s 413us/step - loss: 9.9697e-04 - acc: 0.0027 - val_loss: 5.2537e-04 - val_acc: 0.0000e+00
Epo

734/734 [==============================] - 0s 407us/step - loss: 0.0011 - acc: 0.0027 - val_loss: 5.7001e-04 - val_acc: 0.0000e+00
Epoch 173/300
734/734 [==============================] - 0s 426us/step - loss: 9.8210e-04 - acc: 0.0027 - val_loss: 0.0018 - val_acc: 0.0000e+00
Epoch 174/300
734/734 [==============================] - 0s 403us/step - loss: 0.0011 - acc: 0.0027 - val_loss: 5.2602e-04 - val_acc: 0.0000e+00
Epoch 175/300
734/734 [==============================] - 0s 420us/step - loss: 0.0010 - acc: 0.0027 - val_loss: 7.0697e-04 - val_acc: 0.0000e+00
Epoch 176/300
734/734 [==============================] - 0s 400us/step - loss: 0.0010 - acc: 0.0027 - val_loss: 0.0012 - val_acc: 0.0000e+00
Epoch 177/300
734/734 [==============================] - 0s 411us/step - loss: 0.0010 - acc: 0.0027 - val_loss: 7.0811e-04 - val_acc: 0.0000e+00
Epoch 178/300
734/734 [==============================] - 0s 405us/step - loss: 0.0011 - acc: 0.0027 - val_loss: 6.3799e-04 - val_acc: 0.0000e+00
Epo

Epoch 229/300
734/734 [==============================] - 0s 444us/step - loss: 0.0014 - acc: 0.0027 - val_loss: 9.7855e-04 - val_acc: 0.0000e+00
Epoch 230/300
734/734 [==============================] - 0s 423us/step - loss: 9.9362e-04 - acc: 0.0027 - val_loss: 0.0013 - val_acc: 0.0000e+00
Epoch 231/300
734/734 [==============================] - 0s 426us/step - loss: 0.0011 - acc: 0.0027 - val_loss: 0.0018 - val_acc: 0.0000e+00
Epoch 232/300
734/734 [==============================] - 0s 424us/step - loss: 9.7739e-04 - acc: 0.0027 - val_loss: 5.8964e-04 - val_acc: 0.0000e+00
Epoch 233/300
734/734 [==============================] - 0s 421us/step - loss: 0.0011 - acc: 0.0027 - val_loss: 0.0011 - val_acc: 0.0000e+00
Epoch 234/300
734/734 [==============================] - 0s 426us/step - loss: 0.0011 - acc: 0.0027 - val_loss: 0.0017 - val_acc: 0.0000e+00
Epoch 235/300
734/734 [==============================] - 0s 465us/step - loss: 0.0010 - acc: 0.0027 - val_loss: 6.9291e-04 - val_acc: 0.00

734/734 [==============================] - 0s 460us/step - loss: 0.0011 - acc: 0.0027 - val_loss: 0.0011 - val_acc: 0.0000e+00
Epoch 286/300
734/734 [==============================] - 0s 445us/step - loss: 9.8181e-04 - acc: 0.0027 - val_loss: 4.8111e-04 - val_acc: 0.0000e+00
Epoch 287/300
734/734 [==============================] - 0s 432us/step - loss: 9.2838e-04 - acc: 0.0027 - val_loss: 9.1809e-04 - val_acc: 0.0000e+00
Epoch 288/300
734/734 [==============================] - 0s 425us/step - loss: 0.0010 - acc: 0.0027 - val_loss: 6.0927e-04 - val_acc: 0.0000e+00
Epoch 289/300
734/734 [==============================] - 0s 440us/step - loss: 8.8634e-04 - acc: 0.0027 - val_loss: 8.6232e-04 - val_acc: 0.0000e+00
Epoch 290/300
734/734 [==============================] - 0s 454us/step - loss: 9.9362e-04 - acc: 0.0027 - val_loss: 0.0025 - val_acc: 0.0000e+00
Epoch 291/300
734/734 [==============================] - 0s 437us/step - loss: 0.0014 - acc: 0.0027 - val_loss: 9.4151e-04 - val_acc: 0.

KeyError: 'close'

In [12]:
import pandas as pd

foxconndf= pd.read_csv('./0050.csv', index_col=0 )
foxconndf.dropna(how='any',inplace=True)
# print(type(foxconndf))
# print(foxconndf)
foxconndf_norm= normalize(foxconndf)
#print(foxconndf_norm)

number_features = len(foxconndf.columns)
print(number_features)
datavalue = foxconndf.as_matrix()
print(len(datavalue))

71
913


C:\Users\Big data\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [14]:
for index in range( len(datavalue) - (5+1) ): 
    print(datavalue[index: index + (5+1) ]) 

[[ 2.01404280e+07  5.32300000e+01  5.42900000e+01  5.32300000e+01
   5.40700000e+01  8.48200000e+03  1.06000000e+00  8.40000000e-01
   3.84105960e+01  4.91150442e+01  5.37931034e+01  6.26749611e+01
   5.67164179e+01  5.68145800e+01  5.59227844e+01  5.44114833e+01
   5.48622366e+01  5.40173053e+01  5.40533333e+01  5.42500000e+01
   5.41558333e+01  5.35611538e+01  5.23008333e+01  5.19111111e+01
   5.14806667e+01  5.07045556e+01  5.01865000e+01  4.96688000e+01
   5.39800000e+01  5.41685714e+01  5.41801282e+01  5.39427350e+01
   5.30260874e+01  5.25121856e+01  5.21431322e+01  5.15848122e+01
   5.10556559e+01  5.06406441e+01  5.35800000e+01  5.36262851e+01
   5.36721717e+01  5.37176839e+01  5.37628113e+01  5.38075004e+01
   5.38516437e+01  5.38950649e+01  5.39375001e+01  5.39785717e+01
   5.40177477e+01  5.40542727e+01  5.40870337e+01  5.41142736e+01
   5.41329294e+01  5.41370146e+01  5.41135019e+01  5.40311644e+01
   5.38056050e+01  5.31555976e+01  5.39150000e+01  5.41935484e+01
   3.53939

   3.78205128e+01  1.52285953e+01  2.23313095e+01]]
[[ 2.01409290e+07  5.79500000e+01  5.80000000e+01  5.75600000e+01
   5.77300000e+01  1.71790000e+04  4.40000000e-01 -2.20000000e-01
   0.00000000e+00  5.34979424e+00  1.65501166e+01  3.86214442e+01
   5.12820513e+01  5.58179329e+01  5.74281806e+01  5.66739606e+01
   5.62508809e+01  5.50681431e+01  5.79533333e+01  5.86728571e+01
   5.91091667e+01  6.00757692e+01  5.97048333e+01  5.87322222e+01
   5.76086667e+01  5.56190000e+01  5.43378333e+01  5.32287333e+01
   5.78716667e+01  5.83146429e+01  5.87473077e+01  5.95617664e+01
   5.96878197e+01  5.94293480e+01  5.88435937e+01  5.75327931e+01
   5.63803800e+01  5.54307161e+01  5.79100000e+01  5.79268536e+01
   5.79465422e+01  5.79693021e+01  5.79954546e+01  5.80254335e+01
   5.80598095e+01  5.80993101e+01  5.81448401e+01  5.81975090e+01
   5.82586865e+01  5.83301174e+01  5.84141579e+01  5.85142234e+01
   5.86355502e+01  5.87862147e+01  5.89774288e+01  5.92184743e+01
   5.94890876e+01  5.954

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[[ 2.01612150e+07  6.85000000e+01  6.89200000e+01  6.83500000e+01
   6.89200000e+01  3.01700000e+03  5.70000000e-01  4.20000000e-01
   7.05882353e+01  8.50980392e+01  6.54450262e+01  5.24187153e+01
   5.22014676e+01  5.46711153e+01  5.81233244e+01  5.61260494e+01
   5.49428255e+01  5.46039149e+01  6.89366667e+01  6.87800000e+01
   6.82883333e+01  6.75600000e+01  6.77175000e+01  6.71457778e+01
   6.61567500e+01  6.35858889e+01  6.17727083e+01  6.10511333e+01
   6.89450000e+01  6.88875000e+01  6.85882051e+01  6.80158689e+01
   6.77749672e+01  6.75451624e+01  6.71855331e+01  6.57604395e+01
   6.44153354e+01  6.33043014e+01  6.90200000e+01  6.90123083e+01
   6.90043484e+01  6.89962376e+01  6.89879897e+01  6.89794736e+01
   6.89703647e+01  6.89600890e+01  6.89477607e+01  6.89321143e+01
   6.89114326e+01  6.88834635e+01  6.88453044e+01  6.87931860e+01
   6.87220134e+01  6.86244487e+01  6.84896357e+01  6.83037582e+01
   6.80595436e+01  6.77006569e+01  6.87775000e+01  5.22935780e+01
   5.30672

   6.99792961e+01  8.69198876e+01  8.50052553e+01]]
[[ 2.01706280e+07  7.81900000e+01  7.83400000e+01  7.76100000e+01
   7.78600000e+01  6.28600000e+03  7.30000000e-01 -3.30000000e-01
   6.68016194e+01  7.63586957e+01  7.83132530e+01  7.19806763e+01
   6.47743814e+01  6.33516306e+01  6.24813154e+01  5.89265046e+01
   5.97568579e+01  5.75575250e+01  7.83733333e+01  7.75585714e+01
   7.64341667e+01  7.54969231e+01  7.37001667e+01  7.27476667e+01
   7.17372500e+01  7.03996667e+01  6.91504583e+01  6.70341000e+01
   7.82366667e+01  7.78510714e+01  7.71871795e+01  7.61606268e+01
   7.47531148e+01  7.38721270e+01  7.31546088e+01  7.19150909e+01
   7.09807566e+01  6.99115395e+01  7.85800000e+01  7.85808800e+01
   7.85735290e+01  7.85579285e+01  7.85339959e+01  7.85015424e+01
   7.84602055e+01  7.84093509e+01  7.83479384e+01  7.82743504e+01
   7.81861832e+01  7.80800136e+01  7.79511531e+01  7.77933853e+01
   7.75985960e+01  7.73558950e+01  7.70488095e+01  7.66451897e+01
   7.60544855e+01  7.487

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
len(datavalue[1])

71

In [17]:
len(datavalue)

913

In [31]:
result=[]
for index in range( len(datavalue) - (5+1) ): 
    result.append(datavalue[index: index + (5+1) ]) 

result = np.array(result)
number_train = round(0.9 * result.shape[0]) 
x_train = result[:int(number_train), :-1] 
y_train = result[:int(number_train), -1][:,-1]
print(len(x_train[0]))
print(len(y_train[0]))

5


TypeError: object of type 'numpy.float64' has no len()